In [ ]:
!pip install transformers

In [ ]:
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
from IPython.display import clear_output
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
import plotly.express as px
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/df_difficult_26_01.csv')

In [ ]:
def get_difficult(text):
  return text[100:-100]

df['text_dif_bert'] = df['doc_text'].apply(get_difficult)

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
tokenized_texts = df['doc_text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True)).values

max_len = max(len(text) for text in tokenized_texts)
padded_texts = torch.tensor([text + [0] * (max_len - len(text)) for text in tokenized_texts])

attention_mask = torch.where(padded_texts > 0, 1, 0)

In [ ]:
tokenized_texts_dif = df['text_dif_bert'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True)).values

max_len = max(len(text) for text in tokenized_texts_dif)
padded_texts_dif = torch.tensor([text + [0] * (max_len - len(text)) for text in tokenized_texts_dif])

attention_mask_dif = torch.where(padded_texts_dif > 0, 1, 0)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
model = model_class.from_pretrained(pretrained_weights)
model.to(device)
batch_size = 16
features = []
with torch.no_grad():
    for i in range(0, len(padded_texts), batch_size):
        texts_batch = padded_texts[i : i + batch_size].to(device)
        mask_batch = attention_mask[i : i + batch_size].to(device)
        output = model(texts_batch, mask_batch)
        batch_features = output.last_hidden_state[:, 0, :].cpu().numpy()
        features.append(batch_features)

features = np.concatenate(features, axis=0)
features.shape

(22891, 768)

In [ ]:
model = model_class.from_pretrained(pretrained_weights)
model.to(device)
batch_size = 16
features_dif = []
with torch.no_grad():
    for i in range(0, len(padded_texts), batch_size):
        texts_batch_dif = padded_texts_dif[i : i + batch_size].to(device)
        mask_batch_dif = attention_mask_dif[i : i + batch_size].to(device)
        output_dif = model(texts_batch_dif, mask_batch_dif)
        batch_features_dif = output_dif.last_hidden_state[:, 0, :].cpu().numpy()
        features_dif.append(batch_features_dif)

features_dif = np.concatenate(features_dif, axis=0)
features_dif.shape

(22891, 768)

In [ ]:
labels = df['doc_name'].values

X_train, X_test, y_train, y_test = train_test_split(features, labels,  test_size=0.2, random_state=42)

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300, init='random')
tsne_results = tsne.fit_transform(X_train[:1500])
fig = px.scatter(
    tsne_results, x=0, y=1,
    color=y_train[:1500], labels={'color': 'Вид НПА'}
)
fig.show()

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1500 samples in 0.001s...
[t-SNE] Computed neighbors for 1500 samples in 0.381s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1500
[t-SNE] Computed conditional probabilities for sample 1500 / 1500
[t-SNE] Mean sigma: 0.267574
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.666290
[t-SNE] KL divergence after 300 iterations: 1.587427


In [ ]:
def print_metrics(test, preds):
  print(metrics.classification_report(test, preds))
  print(metrics.accuracy_score(test, preds), 'accuracy')
  print(metrics.precision_score(test, preds, average='macro'), 'precision')
  print(metrics.recall_score(test, preds, average='macro'), 'recall')
  print(metrics.f1_score(test, preds, average='macro'), 'f1')

In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(random_state=42))
])

parameters = {
              'lr_clf__solver':['lbfgs', 'newton-cg', 'sag', 'saga'],
              'lr_clf__class_weight':[None, 'balanced'],
              'lr_clf__penalty':[None, 'l2', 'l1', 'elasticnet']
              }
model = GridSearchCV(lr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:1000], y_train[:1000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.913 {'lr_clf__class_weight': None, 'lr_clf__penalty': None, 'lr_clf__solver': 'newton-cg'}


In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(solver='newton-cg',penalty=None, class_weight=None, random_state=42))
])
lr_ppl_clf.fit(X_train, y_train)
predicted_lr_val = lr_ppl_clf.predict(X_test)
print_metrics(y_test, predicted_lr_val)

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       1.00      0.99      1.00       329
                Закон, федеральный уровень, законодательный орган       1.00      1.00      1.00       347
 Постановление, региональный уровень, глава исполнительной власти       0.98      0.98      0.98       318
       Постановление, региональный уровень, законодательный орган       0.98      0.98      0.98       302
Постановление, региональный уровень, органы исполнительной власти       1.00      0.97      0.98       244
               Постановление, региональный уровень, правительство       0.97      0.97      0.97       318
                Постановление, федеральный уровень, правительство       1.00      1.00      1.00       314
       Приказ, региональный уровень, органы исполнительной власти       0.97      0.98      0.98       325
        Приказ, федеральный уровень,

In [ ]:
svm_ppl_clf = Pipeline([

    ('svm_clf', SVC(random_state=42))
])

parameters = {
              'svm_clf__C': [0.1, 1, 10, 100, 1000],
              'svm_clf__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'svm_clf__kernel': ['rbf']
              }
model = GridSearchCV(svm_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:1000], y_train[:1000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.9239999999999999 {'svm_clf__C': 100, 'svm_clf__gamma': 1, 'svm_clf__kernel': 'rbf'}


In [ ]:
svm_ppl_clf = Pipeline([
    ('svm_clf', SVC(C=100,gamma=1, kernel='rbf', random_state=42))
])
svm_ppl_clf.fit(X_train, y_train)


predicted_svm_val = svm_ppl_clf.predict(X_test)
print_metrics(y_test, predicted_svm_val)

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       1.00      1.00      1.00       329
                Закон, федеральный уровень, законодательный орган       1.00      1.00      1.00       347
 Постановление, региональный уровень, глава исполнительной власти       0.99      0.98      0.98       318
       Постановление, региональный уровень, законодательный орган       0.98      0.99      0.99       302
Постановление, региональный уровень, органы исполнительной власти       1.00      1.00      1.00       244
               Постановление, региональный уровень, правительство       0.98      0.97      0.97       318
                Постановление, федеральный уровень, правительство       1.00      1.00      1.00       314
       Приказ, региональный уровень, органы исполнительной власти       0.97      0.98      0.98       325
        Приказ, федеральный уровень,

In [ ]:
fr_ppl_clf = Pipeline([

    ('rf_clf', RandomForestClassifier(random_state=42))
])

parameters = {
              'rf_clf__n_estimators':[ 300, 500, 700],
              'rf_clf__max_depth':[ 5, 8, 11, 14],
              'rf_clf__criterion':['gini', 'entropy', 'log_loss']
              }
model = GridSearchCV(fr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:1000], y_train[:1000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

In [ ]:
fr_ppl_clf = Pipeline([

    ('sgd_clf', RandomForestClassifier(n_estimators=500,max_depth=14, criterion='gini', random_state=42))
])
fr_ppl_clf.fit(X_train, y_train)


ppredicted_fr_val = fr_ppl_clf.predict(X_test)
print_metrics(y_test, ppredicted_fr_val)

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.96      0.96      0.96       329
                Закон, федеральный уровень, законодательный орган       0.99      1.00      0.99       347
 Постановление, региональный уровень, глава исполнительной власти       0.92      0.88      0.90       318
       Постановление, региональный уровень, законодательный орган       0.97      0.95      0.96       302
Постановление, региональный уровень, органы исполнительной власти       1.00      0.92      0.96       244
               Постановление, региональный уровень, правительство       0.89      0.82      0.86       318
                Постановление, федеральный уровень, правительство       0.98      0.98      0.98       314
       Приказ, региональный уровень, органы исполнительной власти       0.88      0.92      0.90       325
        Приказ, федеральный уровень,

Вывод: использование BERT перед подачей классификатору повышает точность результата.
Также можно предположить, что релевантная для классификации информация содержится в начале текста - что неудивительно, т к это соответствует реальной структуре юридического документа

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_dif, labels,  test_size=0.2, random_state=42)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_dif, labels,  test_size=0.2, random_state=42)
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300, init='random')
tsne_results = tsne.fit_transform(X_train[:1500])
fig = px.scatter(
    tsne_results, x=0, y=1,
    color=y_train[:1500], labels={'color': 'Вид НПА'}
)
fig.show()

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1500 samples in 0.002s...
[t-SNE] Computed neighbors for 1500 samples in 0.148s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1500
[t-SNE] Computed conditional probabilities for sample 1500 / 1500
[t-SNE] Mean sigma: 0.261465
[t-SNE] KL divergence after 50 iterations with early exaggeration: 69.971519
[t-SNE] KL divergence after 300 iterations: 1.723608


In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(random_state=42))
])

parameters = {
              'lr_clf__solver':['lbfgs', 'newton-cg', 'sag', 'saga'],
              'lr_clf__class_weight':[None, 'balanced'],
              'lr_clf__penalty':[None, 'l2', 'l1', 'elasticnet']
              }
model = GridSearchCV(lr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:1000], y_train[:1000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.539 {'lr_clf__class_weight': 'balanced', 'lr_clf__penalty': None, 'lr_clf__solver': 'newton-cg'}


In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(solver='newton-cg',penalty=None, class_weight='balanced', random_state=42))
])
lr_ppl_clf.fit(X_train, y_train)
predicted_lr_val = lr_ppl_clf.predict(X_test)
print_metrics(y_test, predicted_lr_val)

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.89      0.90      0.90       329
                Закон, федеральный уровень, законодательный орган       0.98      0.98      0.98       347
 Постановление, региональный уровень, глава исполнительной власти       0.76      0.75      0.76       318
       Постановление, региональный уровень, законодательный орган       0.85      0.83      0.84       302
Постановление, региональный уровень, органы исполнительной власти       0.89      0.91      0.90       244
               Постановление, региональный уровень, правительство       0.71      0.72      0.72       318
                Постановление, федеральный уровень, правительство       0.91      0.88      0.89       314
       Приказ, региональный уровень, органы исполнительной власти       0.82      0.82      0.82       325
        Приказ, федеральный уровень,

In [ ]:
svm_ppl_clf = Pipeline([

    ('svm_clf', SVC(random_state=42))
])

parameters = {
              'svm_clf__C': [0.1, 1, 10, 100, 1000],
              'svm_clf__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'svm_clf__kernel': ['rbf']
              }
model = GridSearchCV(svm_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:1000], y_train[:1000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.539 {'svm_clf__C': 100, 'svm_clf__gamma': 1, 'svm_clf__kernel': 'rbf'}


In [ ]:
svm_ppl_clf = Pipeline([
    ('svm_clf', SVC(C=100,gamma=1, kernel='rbf', random_state=42))
])
svm_ppl_clf.fit(X_train, y_train)


predicted_svm_val = svm_ppl_clf.predict(X_test)
print_metrics(y_test, predicted_svm_val)

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.89      0.93      0.91       329
                Закон, федеральный уровень, законодательный орган       0.98      0.97      0.97       347
 Постановление, региональный уровень, глава исполнительной власти       0.82      0.81      0.82       318
       Постановление, региональный уровень, законодательный орган       0.90      0.89      0.90       302
Постановление, региональный уровень, органы исполнительной власти       0.95      0.93      0.94       244
               Постановление, региональный уровень, правительство       0.72      0.76      0.74       318
                Постановление, федеральный уровень, правительство       0.90      0.89      0.90       314
       Приказ, региональный уровень, органы исполнительной власти       0.84      0.87      0.85       325
        Приказ, федеральный уровень,

In [ ]:
fr_ppl_clf = Pipeline([

    ('rf_clf', RandomForestClassifier(random_state=42))
])

parameters = {
              'rf_clf__n_estimators':[ 300, 500, 700],
              'rf_clf__max_depth':[ 5, 8, 11, 14],
              'rf_clf__criterion':['gini', 'entropy', 'log_loss']
              }
model = GridSearchCV(fr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:1000], y_train[:1000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.487 {'rf_clf__criterion': 'entropy', 'rf_clf__max_depth': 11, 'rf_clf__n_estimators': 700}


In [ ]:
fr_ppl_clf = Pipeline([

    ('sgd_clf', RandomForestClassifier(n_estimators=700,max_depth=11, criterion='entropy', random_state=42))
])
fr_ppl_clf.fit(X_train, y_train)


ppredicted_fr_val = fr_ppl_clf.predict(X_test)
print_metrics(y_test, ppredicted_fr_val)

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.69      0.78      0.73       329
                Закон, федеральный уровень, законодательный орган       0.91      0.90      0.90       347
 Постановление, региональный уровень, глава исполнительной власти       0.61      0.61      0.61       318
       Постановление, региональный уровень, законодательный орган       0.76      0.69      0.72       302
Постановление, региональный уровень, органы исполнительной власти       0.92      0.86      0.89       244
               Постановление, региональный уровень, правительство       0.58      0.60      0.59       318
                Постановление, федеральный уровень, правительство       0.73      0.74      0.74       314
       Приказ, региональный уровень, органы исполнительной власти       0.60      0.74      0.67       325
        Приказ, федеральный уровень,